In [ ]:
import numpy as np
import scipy as sp
from scipy import stats
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns

plt.rcParams.update({'font.size': 16})

Read in restricted behavioral data:

In [ ]:
res_behav_data = pd.read_csv('../data/RESTRICTED_esfinn_11_21_2021_19_19_35.csv')
res_behav_data.set_index("Subject", inplace=True)
res_behav_data.index = res_behav_data.index.map(str)
print(res_behav_data.shape)
res_behav_data.head()

Read in unrestricted behavioral data:

In [ ]:
unres_behav_data = pd.read_csv('../data/unrestricted_esfinn_11_21_2021_19_19_13.csv')
unres_behav_data.set_index("Subject", inplace=True)
unres_behav_data.index = unres_behav_data.index.map(str)
print(unres_behav_data.shape)
unres_behav_data.head()

Combine restricted and unrestricted behavioral data into a single dataframe:

In [ ]:
behav_data = pd.concat([res_behav_data, unres_behav_data], axis=1)
print(behav_data.shape)
behav_data.head()

Read in FMRI data, join into single dataframe, then join this with the larger dataframe:

In [ ]:
data_file_loc = '/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/fMRIScipts/data'
sub_id_all = np.load(os.path.join(data_file_loc,'sub_ID_all.npy'))
sub_id_all = [str(i) for i in sub_id_all]
print(len(sub_id_all),sub_id_all[:5])

In [ ]:
# load behavioral responses too - to identify missed/invalid trials later - 1 for "social", 0 for "nonsocial" and 9 for "unsure", nan for missed response
responses = np.load(os.path.join(data_file_loc,'responses.npy'))
responses.shape # subs *movies

In [ ]:
coef_slope_rand = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Random mechanical.npy'))
coef_slope_fish = np.load(os.path.join(data_file_loc,'coefs_run_norm','slope_reg','coef_slopereg_runnorm_Fishing.npy'))
coef_slope_coax = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_COAXING-B.npy'))
coef_slope_bill = np.load(os.path.join(data_file_loc,'coefs_trial_norm','slope_reg','coef_slopereg_trialnorm_BILLIARD-A.npy'))
coef_slope_all  = np.load(os.path.join(data_file_loc,'coef_slopereg_all.npy'))

print(coef_slope_rand.shape)
coef_slope_all.shape

# Explore correlations between proportion of "social" responses and various trait phenotypes:

In [ ]:
from multipy.fdr import lsu
corrtype='Spearman' # 'Pearson'


In [ ]:
def explore_correlation(x, y, data,corrtype='Spearman'):
    """
    Calculates and plots correlation between x and y variables in dataframe `data`, plus distribution of x and y 
    """
    sns.set_style("white")
    
    inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) # find rows where neither x or y is NaN

    g = sns.jointplot(x=x, y=y, data=data, kind='reg', color='gray')

    # Calculate and print correlations
    if corrtype == 'Pearson':
        rp, pp = stats.pearsonr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_p = {rp:.2f} (p={pp:.1g})', xy=(.05,.95), xycoords='axes fraction')
    elif corrtype == 'Spearman':
        rs, ps = stats.spearmanr(data[x][inds], data[y][inds])
        g.ax_joint.annotate(f'r_s = {rs:.2f} (p={ps:.1g})', xy=(.05,.95), xycoords='axes fraction')
    
    return g

In [ ]:
# same analysis as above, all nodes
# RANDOM MECH

x = "slopeReg_node"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_rand,   pval_nodes_rand    =  [np.empty((268,2)) for _ in range(2)] # nodes * responses (non-social, social)
corr_nodes_rand[:],pval_nodes_rand[:] = [np.nan] *2

pval_nodes_corr_rand = np.empty_like(pval_nodes_rand)
pval_nodes_corr_rand[:] = np.nan

for n in range(268): 
    if n% 50 ==0:
        print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    for resp in [0,1]:
        d = data.loc[data['responses']==resp,:]
        inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
        
        # Calculate and print correlations
        if corrtype == 'Pearson':
            corr_nodes_rand[n,resp], pval_nodes_rand[n,resp] = stats.pearsonr(d[x][inds], d[y][inds])
        elif corrtype =='Spearman':
            corr_nodes_rand[n,resp], pval_nodes_rand[n,resp] = stats.spearmanr(d[x][inds], d[y][inds])

        pval_nodes_corr_rand[:,resp] = lsu(pval_nodes_rand[:,resp],q=.05)

#print(np.where(pval_nodes_p_corr)[0])
#pval_nodes_sp_corr = lsu(pval_nodes_sp,q=.05)
#print(np.where(pval_nodes_sp_corr)[0])


In [ ]:
resp_lbl

In [ ]:
print('RANDOM MECH')

print('p corr. "Non-social" responders',np.where(pval_nodes_corr_rand[:,0]))
print('p corr. "Social" responders',np.where(pval_nodes_corr_rand[:,1]))

print('p unc. "Non-social" responders',np.where(pval_nodes_rand[:,0]<.05))
print('p unc. "Social" responders',np.where(pval_nodes_rand[:,1]<.05))



In [ ]:
# same analysis as above, all nodes
# FISHING

x = "slopeReg_node"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_fish,   pval_nodes_fish    =  [np.empty((268,2)) for _ in range(2)]
corr_nodes_fish[:],pval_nodes_fish[:] = [np.nan] *2

pval_nodes_corr_fish = np.empty_like(pval_nodes_fish)
pval_nodes_corr_fish[:] = np.nan

for n in range(268): 
    if n% 50 ==0:
        print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_fish[:,n], 'responses':responses[:,3]})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    for resp in [0,1]:
        d = data.loc[data['responses']==resp,:]
        inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
        
        # Calculate and print correlations
        if corrtype == 'Pearson':
            corr_nodes_fish[n,resp], pval_nodes_fish[n,resp] = stats.pearsonr(d[x][inds], d[y][inds])
        elif corrtype =='Spearman':
            corr_nodes_fish[n,resp], pval_nodes_fish[n,resp] = stats.spearmanr(d[x][inds], d[y][inds])

        pval_nodes_corr_fish[:,resp] = lsu(pval_nodes_fish[:,resp],q=.05)

#print(np.where(pval_nodes_p_corr)[0])
#pval_nodes_sp_corr = lsu(pval_nodes_sp,q=.05)
#print(np.where(pval_nodes_sp_corr)[0])


In [ ]:
print('FISHING')

print('p corr. "Non-social" responders',np.where(pval_nodes_corr_fish[:,0]))
print('p corr. "Social" responders',np.where(pval_nodes_corr_fish[:,1]))

print('p unc. "Non-social" responders',np.where(pval_nodes_fish[:,0]<.05))
print('p unc. "Social" responders',np.where(pval_nodes_fish[:,1]<.05))


In [ ]:
# same analysis as above, all nodes
# COAX-BILL

x = "slopeReg_diff"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_coaxbill,   pval_nodes_coaxbill    =  [np.empty((268,)) for _ in range(2)]
corr_nodes_coaxbill[:],pval_nodes_coaxbill[:] = [np.nan] *2

pval_nodes_corr_coaxbill = np.empty_like(pval_nodes_coaxbill)
pval_nodes_corr_coaxbill[:] = np.nan

for n in range(268): 
    if n% 50 ==0:
        print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_diff':coef_slope_coax[:,n]-coef_slope_bill[:,n], 'responses':(responses[:,0]==1)&(responses[:,1]==0)})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    d = data.loc[data['responses']==True,:]
    inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
    
    # Calculate and print correlations
    if corrtype == 'Pearson':
        corr_nodes_coaxbill[n], pval_nodes_coaxbill[n] = stats.pearsonr(d[x][inds], d[y][inds])
    elif corrtype =='Spearman':
        corr_nodes_coaxbill[n], pval_nodes_coaxbill[n] = stats.spearmanr(d[x][inds], d[y][inds])

    pval_nodes_corr_coaxbill = lsu(pval_nodes_coaxbill,q=.05)

#print(np.where(pval_nodes_p_corr)[0])
#pval_nodes_sp_corr = lsu(pval_nodes_sp,q=.05)
#print(np.where(pval_nodes_sp_corr)[0])/


In [ ]:
print('COAX-BILL')
print('p corr. "Social"- "Non-social" responders',np.where(pval_nodes_corr_coaxbill))
print('p unc. "Social" - "Non-social" responders',np.where(pval_nodes_coaxbill<.05))

In [ ]:
# same analysis as above, all nodes
# ALL MOVIES

x = "slopeReg_diff"
#y = "Loneliness_Unadj"
y = "ASR_Intn_T"

corr_nodes_all,   pval_nodes_all    =  [np.empty((268,)) for _ in range(2)]
corr_nodes_all[:],pval_nodes_all[:] = [np.nan] *2

pval_nodes_corr_all = np.empty_like(pval_nodes_all)
pval_nodes_corr_all[:] = np.nan

mean_coef_slope_all = np.nanmean(coef_slope_all,axis=2)
mean_coef_slope_all.shape

nresp = [len(np.where(~np.isnan(responses[i,:]))[0]) for i in range(responses.shape[0])]
for n in range(268):
    if n% 50 ==0:
        print('node',n)

    fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_diff':mean_coef_slope_all[:,n], 'responses':nresp})
    fMRI_data.set_index("Subject", inplace=True)
    data = fMRI_data.join(behav_data, how='inner') # join betas and trait info
    #if n == 0:
    #    print(data.shape)
    #    print(data.head())
    
    d = data.loc[data['responses']==10,:]
    inds = ~np.isnan(d[x]) & ~np.isnan(d[y]) # find rows where neither x or y is NaN
    
    # Calculate and print correlations
    if corrtype == 'Pearson':
        corr_nodes_all[n], pval_nodes_all[n] = stats.pearsonr(d[x][inds], d[y][inds])
    elif corrtype =='Spearman':
        corr_nodes_all[n], pval_nodes_all[n] = stats.spearmanr(d[x][inds], d[y][inds])

    pval_nodes_corr_all = lsu(pval_nodes_all,q=.05)

#print(np.where(pval_nodes_p_corr)[0])
#pval_nodes_sp_corr = lsu(pval_nodes_sp,q=.05)
#print(np.where(pval_nodes_sp_corr)[0])


In [ ]:
len(nresp)

In [ ]:
nsubs = {'RAND':[len(np.where(responses[:,4]==0)[0]),len(np.where(responses[:,4]==1)[0])],
'FISH':[len(np.where(responses[:,3]==0)[0]),len(np.where(responses[:,3]==1)[0])],
'COAXBILL':len(np.where((responses[:,0]==1) & (responses[:,1]==0))[0]),
'ALL':len(np.where(np.array(nresp)==10)[0])}
nsubs

In [ ]:
print('ALL MOVIES')

print('p corr. "Social"- "Non-social" responders',np.where(pval_nodes_corr_all))
print('p unc. "Social" - "Non-social" responders',np.where(pval_nodes_all<.05))

In [ ]:
# What are the ranges of these correlations?

for resp in [0,1]:
    nmin,nmax = np.argmin(corr_nodes_rand[:,resp]),np.argmax(corr_nodes_rand[:,resp])
    print(f'\n\nRANDOM MECH\nresp:{resp}, min node={nmin+1}, rSp={corr_nodes_rand[nmin,resp]:.2f}, pSp = {pval_nodes_rand[nmin,resp]:.2e}')
    print(f'resp:{resp}, max node={nmax+1}, rSp={corr_nodes_rand[nmax,resp]:.2f}, pSp = {pval_nodes_rand[nmax,resp]:.2e}')

    nmin,nmax = np.argmin(corr_nodes_fish[:,resp]),np.argmax(corr_nodes_fish[:,resp])
    print(f'\n\nFISHING\nresp:{resp}, min node={nmin+1}, rSp={corr_nodes_fish[nmin,resp]:.2f}, pSp = {pval_nodes_fish[nmin,resp]:.2e}')
    print(f'resp:{resp}, max node={nmax+1}, rSp={corr_nodes_fish[nmax,resp]:.2f}, pSp = {pval_nodes_fish[nmax,resp]:.2e}')

In [ ]:
nmin,nmax = np.argmin(corr_nodes_coaxbill),np.argmax(corr_nodes_coaxbill)
print(f'coaxbill, Node {nmin+1}, rSp={corr_nodes_coaxbill[nmin]:.2f}, pSp = {pval_nodes_coaxbill[nmin]:.2e}')
print(f'coaxbill, Node {nmax+1}, rSp={corr_nodes_coaxbill[nmax]:.2f}, pSp = {pval_nodes_coaxbill[nmax]:.2e}')


nmin,nmax = np.argmin(corr_nodes_all),np.argmax(corr_nodes_all)
print(f'\n\nall movies, Node {nmin+1}, rSp={corr_nodes_coaxbill[nmin]:.2f}, pSp = {pval_nodes_coaxbill[nmin]:.2e}')
print(f'all movies, Node {nmax+1}, rSp={corr_nodes_coaxbill[nmax]:.2f}, pSp = {pval_nodes_coaxbill[nmax]:.2e}')

In [ ]:
shen268_lbl = pd.read_csv(os.path.join(data_file_loc,"shen_dictionary.csv"))
shen268_lbl.head()

In [ ]:
nodes_ind = np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_fish[:,1]<.05))[0] #& (pval_nodes_coaxbill<.05) & (pval_nodes_all<.05))[0]
print(nodes_ind)
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']

print(lbl)

In [ ]:
nodes_ind = np.where((pval_nodes_coaxbill<.05) & (pval_nodes_all<.05))[0]
print(nodes_ind)
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']

print(lbl)

In [ ]:
# for comparison with the other main figure if needed
nodes_coaxbill_rand_all = np.load(os.path.join(data_file_loc,'nodes_coaxbill_rand_all.npy')) # sig nodes hihglighted in the first S>NS GLM
len(np.where(nodes_coaxbill_rand_all)[0])

# Plot brain maps

In [ ]:
from nltools.data import Brain_Data
from nltools.mask import expand_mask, roi_to_brain
from nilearn.plotting import plot_glass_brain, plot_surf_roi,plot_stat_map,plot_img,plot_surf_contours
from nilearn import datasets
from nilearn.surface import vol_to_surf

import nibabel as nib

from nilearn.datasets import fetch_surf_fsaverage
fsaverage = fetch_surf_fsaverage()

#coords = [10*int(i) for i in np.linspace(-4,7,6)]
bg_img = datasets.load_mni152_template()

mask = Brain_Data('https://neurovault.org/media/images/8423/shen_2mm_268_parcellation.nii.gz')
mask_x = expand_mask(mask)


In [ ]:
def color_rois(values):
    
    """
    This function assumes you are passing a vector "values" with the same length as the number of nodes in the atlas.
    """
    shen268 = nib.load(os.path.join(data_file_loc,"shen_2mm_268_parcellation.nii.gz"))
    shen268_data = shen268.get_fdata()
    img = np.zeros(shen268_data.shape)
    #print(shen268_data.shape)
    for roi in range(len(values)):
        itemindex = np.where(shen268_data==roi+1) # find voxels in this node (add 1 to account for zero-indexing)
        #print(len(itemindex[0]))
        img[itemindex] = values[roi] # color them by the desired value 

    affine = shen268.affine
    img_nii = nib.Nifti1Image(img, affine)
    
    return img_nii

In [ ]:
plt.subplot(221)
nodes = pval_nodes_rand[:,1] <.05
plt.hist(corr_nodes_rand[nodes,1])
plt.title('RANDOM MECH')
plt.xlim(-.25,.25)
plt.ylim(0,20)

plt.subplot(222)
nodes = pval_nodes_fish[:,0] <.05
plt.hist(corr_nodes_fish[nodes,0])
plt.title('FISHING')
plt.xlim(-.25,.25)
plt.ylim(0,20)

plt.subplot(223)
plt.hist(corr_nodes_coaxbill[pval_nodes_coaxbill<.05])
plt.title('COAX-BILL')
plt.xlim(-.25,.25)
plt.ylim(0,20)

plt.subplot(224)
plt.hist(corr_nodes_all[pval_nodes_all<.05])
plt.title('ALL MOVIES')
plt.xlim(-.25,.25)
plt.ylim(0,20)

plt.tight_layout()

In [ ]:
fig_save_loc = os.path.join('/Users/f0053cz/Dropbox (Dartmouth College)/postdoc_Dartmouth/HCP/paper_prep/figures/fig2_2_traits/fmri_results')

In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version

txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  \n  'r"$\beta(''Social''-'$""\n  ''Non-social'')}"
title_txt = 'ALL MOVIES'

nodes = np.zeros((268,))
nodes[pval_nodes_all<.05] = corr_nodes_all[pval_nodes_all<.05]
#vmin,vmax = -max(abs(nodes)),max(abs(nodes))
vmin,vmax = -.32,.32
cmap = 'RdBu_r'

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
#ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()

plt.savefig(os.path.join(fig_save_loc,f'ALL/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot1.axes[1].text(1,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
#LH
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot1, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'ALL/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot2, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'ALL/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot3, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'ALL/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot4.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot4.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot4, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'ALL/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')


In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version

txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  \n  'r"$\overline{\beta(''Social''-}$" + '\n  ' + r"$\overline{\beta(''Non-social'')}$"
title_txt = 'COAX-BILL'

nodes = np.zeros((268,))
nodes[pval_nodes_coaxbill<.05] = corr_nodes_coaxbill[pval_nodes_coaxbill<.05]
#vmin,vmax = -max(abs(nodes)),max(abs(nodes))
vmin,vmax = -.32,.32

cmap = 'RdBu_r'

img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
#ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
#ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

#LH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot1.axes[1].text(1,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot1, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_left, view = 'medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot2, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot3, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right, view ='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
#surf_plot4.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
#surf_plot4.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot4, legend=True,levels = [1], colors=['k'])
plt.savefig(os.path.join(fig_save_loc,f'COAXBILL/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')


In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'

resp_lbl = ["Nonsocial",  "Social"]
    
vmin,vmax = -.32,.32
for resp in [0,1]:
    if resp == 1:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Social'')$"
    else:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Non-social'')$"

    nodes = np.zeros((268,))
    nodes[pval_nodes_rand[:,resp]<.05] = corr_nodes_rand[pval_nodes_rand[:,resp]<.05,resp]

    title_txt = f'RANDOM MECH "{resp_lbl[resp]}"'

    img = roi_to_brain(pd.Series(nodes), mask_x)
    ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
    #ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
    #ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
    plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

    #LH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot1.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot1, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   


    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot2, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

    #RH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot3, legend=True,levels = [1], colors=['k'])
    #surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')


    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot4.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot4.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot4, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'RAND/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')



In [ ]:
coords = [-50,-40,-25,-10,0,10,25,40] # initial exploration
#coords = [-40,-30,-20] # for the final version
cmap = 'RdBu_r'

resp_lbl = ["Nonsocial",  "Social"]

vmin,vmax = -.32,.32
for resp in [0,1]:
    if resp == 1:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Social'')$"
    else:
        txt = f'  {corrtype} r\n  (p<.05 unc.)\n {y} vs  'r"$\overline{\beta}(''Non-social'')$"

    nodes = np.zeros((268,))
    nodes[pval_nodes_fish[:,resp]<.05] = corr_nodes_fish[pval_nodes_fish[:,resp]<.05,resp]

    title_txt = f'FISHING "{resp_lbl[resp]}"'
    
    img = roi_to_brain(pd.Series(nodes), mask_x)
    ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= False,threshold = .0001)
    #ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
    #ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()
    plt.savefig(os.path.join(fig_save_loc,f'FISH/{resp_lbl[resp]}/axial.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

    #LH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    surf_plot1=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot1.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    surf_plot1.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_left,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot1, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'FISH/{resp_lbl[resp]}/left_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')   


    surf_plot2=plot_surf_roi(fsaverage.infl_left, texture, hemi='left',cmap = cmap, colorbar=False, symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_left, view = 'medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot2.axes[1].text(3,.8*vmax,s= txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot2.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plot_surf_contours(fsaverage.infl_left, texture_contour, figure=surf_plot2, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'FISH/{resp_lbl[resp]}/left_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')

    #RH
    texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    texture_contour = vol_to_surf(color_rois(nodes_coaxbill_rand_all), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
    plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot3, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'FISH/{resp_lbl[resp]}/right_lat.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')


    surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=False,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                                bg_map=fsaverage.sulc_right, view ='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
    #surf_plot4.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
    #surf_plot4.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')
    plot_surf_contours(fsaverage.infl_right, texture_contour, figure=surf_plot4, legend=True,levels = [1], colors=['k'])
    plt.savefig(os.path.join(fig_save_loc,f'FISH/{resp_lbl[resp]}/right_med.png'),dpi=300,bbox_inches='tight',facecolor='white', edgecolor='none')


# Plot correlations

In [ ]:
#"Social" alone
nodes_ind = np.where( (pval_nodes_rand[:,1]<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,lbl)

In [ ]:
#"Social" alone
nodes_ind = np.where( (pval_nodes_coaxbill<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,np.round(corr_nodes_coaxbill[n],2),np.round(np.nanmean(coef_slope_coax[:,n]-coef_slope_bill[:,n]),2),lbl)

In [ ]:
y

In [ ]:
x = "slopeReg_diff"

#for n in range(268):
n=138
#n=191 # left MTG
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split(';'))
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_diff':coef_slope_coax[:,n]-coef_slope_bill[:,n], 'responses':(responses[:,0]==1)&(responses[:,1]==0)})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info


d = data.loc[data['responses']==True,:]
g = explore_correlation(x,y,d)
plt.xlabel('Beta COAX-BILL')
plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
plt.xlim(min(data[x]),max(data[x]))
plt.ylim(min(data[y]),max(data[y]))
plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
#g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
np.where(pval_nodes_fish[:,0]<.05)[0]

In [ ]:
np.where(pval_nodes_fish[:,1]<.05)[0]

In [ ]:
np.where(pval_nodes_rand[:,1]<.05)

In [ ]:
nodes_ind =np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_fish[:,0]<.05))[0]
for n in nodes_ind:
    lbl = eval(shen268_lbl[str(n+1)][0])['name']
    print(n+1,lbl)

In [ ]:
np.where((pval_nodes_rand[:,1]<.05) & (pval_nodes_fish[:,1]<.05))

In [ ]:
x = "slopeReg_node"

#for n in range(268):
n=84 
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split())
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info

for resp in [0,1]:
    d = data.loc[data['responses']==resp,:]
    g = explore_correlation(x,y,d,'Spearman')
    plt.xlabel('Beta RANDOM MECH\n(<<< Deactivation   Activation >>>)')
    plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
    plt.xlim(min(data[x]),max(data[x]))
    plt.ylim(min(data[y]),max(data[y]))
    if resp ==0:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
    else:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}"',fontweight='bold')
    #g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
x = "slopeReg_node"
#for n in range(268):
n=84
hemi_txt = 'Right hem' if n < 134 else 'Left hem'
lbl  = '\n'.join(eval(shen268_lbl[str(n+1)][0])['name'].split())
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'slopeReg_node':coef_slope_fish[:,n], 'responses':responses[:,3]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info

for resp in [0,1]:
    d = data.loc[data['responses']==resp,:]
    g = explore_correlation(x,y,d,'Spearman')
    plt.xlabel('Beta FISHING\n(<<< Deactivation   Activation >>>)')
    plt.text(plt.xlim()[0],100,f'"{resp_lbl[resp]}" (n={d.shape[0]}),\n Node{n+1}',fontweight = 'bold')
    plt.xlim(min(data[x]),max(data[x]))
    plt.ylim(min(data[y]),max(data[y]))
    if resp ==0:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}",{hemi_txt}\n{lbl}',fontweight='bold')
    else:
        plt.text(plt.xlim()[0],120,f'"{resp_lbl[resp]}"',fontweight='bold')
    #g.savefig(f'../results/RandBeta{n+1}_vs_loneliness.png')



In [ ]:
nodes = np.zeros((268,))
nodes[n] = corr_nodes[n,1]
cmap = 'RdBu_r'
img = roi_to_brain(pd.Series(nodes), mask_x)
ax_plot = plot_img(img.to_nifti(), display_mode = 'z',vmin = vmin, vmax = vmax, cut_coords =coords,cmap = cmap, bg_img = bg_img,colorbar= True,threshold = .0001)
ax_plot.title(title_txt,fontsize=24,bgcolor='k',color='w',fontweight='bold')
ax_plot._colorbar_ax.text(1,.8*vmax,txt,fontsize=20,fontdict = {'verticalalignment':'top','rotation':0})#get_legend()

#RH
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')

surf_plot4=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right,view='medial')#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})


  

In [ ]:
nodes = np.zeros((268,))
nodes[pval_nodes[:,1]<.05] = corr_nodes[pval_nodes[:,1]<.05,1]
    
texture = vol_to_surf(color_rois(nodes), fsaverage.pial_right,interpolation='nearest',radius =1, n_samples=1)
surf_plot3=plot_surf_roi(fsaverage.infl_right, texture, hemi='right',cmap = cmap, colorbar=True,symmetric_cmap=True, vmin = vmin, vmax = vmax,
                            bg_map=fsaverage.sulc_right)#,vol_to_surf_kwargs={"n_samples": 10, "radius": 10, "interpolation": "nearest","kind":'ball'})
surf_plot3.axes[1].text(3,.8*vmax,s=txt,fontsize=20, fontdict = {'verticalalignment':'top','horizontalalignment':'left','rotation':0})
surf_plot3.axes[0].set_title(title_txt,fontsize=24,color='k',fontweight='bold')

   

# plot timecourses

In [ ]:
data_file_loc

In [ ]:
tr0_ind = 3 # ind at which tr=0 starts
[timepts_indiv_movie,vid_start_rel_tr] = np.load(os.path.join(data_file_loc,'Video_TRs.npy'),allow_pickle=True)
print(vid_start_rel_tr)
l_task = 28

def remove_pretrial_TRs(tcs,vid_start_rel_tr,pretrial_TRs):

    tcs1 = np.empty((tcs.shape[0],tcs.shape[1],28+pretrial_TRs,tcs.shape[3]))
    tcs1[:] = np.nan

    for vid_no in range(10):
        tr0 = vid_start_rel_tr[vid_no]
        tcs1[:,:,:,vid_no] = tcs[:,:,tr0-pretrial_TRs:tr0+28,vid_no] # 31 timepts
    return tcs1

In [ ]:
#select subs who responded "social" to the yes movie and nonsocial to the no movie

print('\ntrial-wise norm data.')
fileName = os.path.join(data_file_loc,'timecourses_run_norm','timecourse-all-movies_zscorenorm.npy') # load fmri data
tcs_run_z = np.load(fileName) # nsubs * nnodes * ntimepts *nmovies
print('before:',tcs_run_z.shape)
tcs_run_z = remove_pretrial_TRs(tcs_run_z,vid_start_rel_tr,tr0_ind)
print('after:',tcs_run_z.shape)

In [ ]:
nodes = np.where(pval_nodes[:,1]<.05)[0]
nodes

In [ ]:
# get the right red/blue shades
#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

In [ ]:
fMRI_data = pd.DataFrame({'Subject':sub_id_all,'sub_ind':np.arange(0,1048),'slopeReg_node':coef_slope_rand[:,n], 'responses':responses[:,4]})
fMRI_data.set_index("Subject", inplace=True)
data = fMRI_data.join(behav_data, how='inner') # join betas and trait info


In [ ]:
fMRI_data


In [ ]:
sorted_data = data.loc[data['responses']==1,['slopeReg_node','ASR_Intn_T','sub_ind','responses']]
sorted_data = sorted_data.sort_values(by=['ASR_Intn_T'])
sorted_data

In [ ]:
movie_ind = 4
ymin,ymax = -5,5
n = 58
subs = sorted_data['sub_ind'].values

ncols = 15
nrows= int(np.ceil(len(subs)/ncols))
#fig, axes = plt.subplots(67,4, figsize=(15,150))
xwidth = 30
yheight = 20# .75*nrows*xwidth/ncols
fig, ax = plt.subplots(nrows, ncols, sharex=True, sharey=True, figsize = (xwidth,yheight))
fig.set_figwidth(xwidth)
fig.set_figheight(yheight)

TR=.72
xlbls = [-2,0,5,10,15,20]
x_ticks = [(i/TR)+tr0_ind for i in xlbls]
xlbls = [str(i) for i in xlbls]
lbls = [str(np.round((i-tr0_ind)*TR,1)) for i in x_ticks]

plt.suptitle(f'Node {n+1},{eval(shen268_lbl[str(n+1)][0])["name"]}',fontweight='bold')

for i,s in enumerate(subs):

    r = int(np.floor(i/ncols))
    c = int(np.floor(i%ncols))

    movieTC_yes = tcs_run_z[s,n,:,movie_ind] # nsubs * ntimepts
    #movieTC_no  = tcs_run_z[responses[:,movie_ind]==0,n,:,movie_ind] # nsubs * ntimepts
    #sig = np.where(~np.isnan(meanDiff_rand[n,:]))[0]    

    ax[r,c].set_ylim(ymin,ymax)
    ax[r,c].plot(range(movieTC_yes.shape[0]),movieTC_yes,color = 'k',linewidth=2,label ='"Social"')
    #ax[r,c].errorbar(range(movieTC_yes.shape[1]),np.nanmean(movieTC_no,axis=0),stats.sem(movieTC_no,axis=0,nan_policy='omit'),color = blue_rgb,linewidth=2,label ='"Non-social"')
    #if r==0:
    if sorted_data["slopeReg_node"][i] > 1:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color='r',fontsize = 16)
    elif sorted_data["slopeReg_node"][i] > .5:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color=[.5,.25,.25],fontsize = 16,)
    elif sorted_data["slopeReg_node"][i] < -1:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color='b',fontsize = 16)
    elif sorted_data["slopeReg_node"][i] < -.5:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}', color=[.25,.25,.5],fontsize = 16)
    else:
        ax[r,c].set_title(f'Sub {s},\nasr={int(sorted_data["ASR_Intn_T"][i])},b={sorted_data["slopeReg_node"][i]:.2f}',fontsize = 16)
    
    ax[r,c].vlines(tr0_ind,-1,2,color=[.8,.8,.8])
    ax[r,c].hlines(0,0,31,color=[.8,.8,.8])
    ax[r,c].set_xticks(x_ticks)
    ax[r,c].set_xticklabels(xlbls)

ax[r,c+1].axis('off')
ax[r,c+2].axis('off')

plt.tight_layout()

In [ ]:
sorted_data['ASR_Intn_T'][i]

In [ ]:
y = "ASR_Extn_T"
g = explore_correlation(x, y)
plt.xlabel('Beta Rand')


Determine whether the difference in correlation between `Soc-NonSoc_pc` and internalizing vs externalizing symptoms is statistically significant:

In [ ]:
x = "Soc-NonSoc_pc"
y = "ASR_Intn_T"
z = "ASR_Extn_T"

inds = ~np.isnan(data[x]) & ~np.isnan(data[y]) & ~np.isnan(data[z])
print(inds.sum())

rs_xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
rs_xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
rs_yz = stats.spearmanr(data[y][inds], data[z][inds])[0]
print(rs_xy)
print(rs_xz)
print(rs_yz)

The following function was taken from here: https://github.com/psinger/CorrelationStats/blob/master/corrstats.py

In [ ]:
from scipy.stats import t, norm
from math import atanh, pow
from numpy import tanh

def rz_ci(r, n, conf_level = 0.95):
    zr_se = pow(1/(n - 3), .5)
    moe = norm.ppf(1 - (1 - conf_level)/float(2)) * zr_se
    zu = atanh(r) + moe
    zl = atanh(r) - moe
    return tanh((zl, zu))

def rho_rxy_rxz(rxy, rxz, ryz):
    num = (ryz-1/2.*rxy*rxz)*(1-pow(rxy,2)-pow(rxz,2)-pow(ryz,2))+pow(ryz,3)
    den = (1 - pow(rxy,2)) * (1 - pow(rxz,2))
    return num/float(den)

def dependent_corr(xy, xz, yz, n, twotailed=True, conf_level=0.95, method='steiger'):
    """
    Calculates the statistic significance between two dependent correlation coefficients
    @param xy: correlation coefficient between x and y
    @param xz: correlation coefficient between x and z
    @param yz: correlation coefficient between y and z
    @param n: number of elements in x, y and z
    @param twotailed: whether to calculate a one or two tailed test, only works for 'steiger' method
    @param conf_level: confidence level, only works for 'zou' method
    @param method: defines the method uses, 'steiger' or 'zou'
    @return: t and p-val
    """
    if method == 'steiger':
        d = xy - xz
        determin = 1 - xy * xy - xz * xz - yz * yz + 2 * xy * xz * yz
        av = (xy + xz)/2
        cube = (1 - yz) * (1 - yz) * (1 - yz)

        t2 = d * np.sqrt((n - 1) * (1 + yz)/(((2 * (n - 1)/(n - 3)) * determin + av * av * cube)))
        p = 1 - t.cdf(abs(t2), n - 3)

        if twotailed:
            p *= 2

        return t2, p
    elif method == 'zou':
        L1 = rz_ci(xy, n, conf_level=conf_level)[0]
        U1 = rz_ci(xy, n, conf_level=conf_level)[1]
        L2 = rz_ci(xz, n, conf_level=conf_level)[0]
        U2 = rz_ci(xz, n, conf_level=conf_level)[1]
        rho_r12_r13 = rho_rxy_rxz(xy, xz, yz)
        lower = xy - xz - pow((pow((xy - L1), 2) + pow((U2 - xz), 2) - 2 * rho_r12_r13 * (xy - L1) * (U2 - xz)), 0.5)
        upper = xy - xz + pow((pow((U1 - xy), 2) + pow((xz - L2), 2) - 2 * rho_r12_r13 * (U1 - xy) * (xz - L2)), 0.5)
        return lower, upper
    else:
        raise Exception('Wrong method!')

In [ ]:
n = inds.sum()

xy = stats.spearmanr(data[x][inds], data[y][inds])[0]
xz = stats.spearmanr(data[x][inds], data[z][inds])[0]
yz = stats.spearmanr(data[y][inds], data[z][inds])[0]

dependent_corr(xy, xz, yz, n, twotailed=False)

Look at responses on RANDMECH:

In [ ]:
all_trial_data = pd.read_csv('../data/hcp_social_826subs.csv')
all_trial_data.set_index("subj_idx", inplace=True)
all_trial_data.index.rename("Subject", inplace=True)
all_trial_data.index = all_trial_data.index.map(str)
all_trial_data.head()

In [ ]:
data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["response"])
data.rename(columns={"response": "rand_mech_response"}, inplace=True)
data.head()

In [ ]:
#red_rgb = [103,0,31] # edges of RdBu
#blue_rgb = [5,48,97] # edges of RdBu
red_rgb =[188,61,62] # from Emily
blue_rgb = [54,122,177] # from Emily
red_rgb = np.array(red_rgb)/255
blue_rgb = np.array(blue_rgb)/255
alpha = .2 # transparency inside boxplots, for datapts etc.

In [ ]:
#colors = [red_rgb,blue_rgb]
# Set your custom color palette
#myPalette = sns.set_palette(sns.color_palette(colors))

In [ ]:
list(col[:2])+[alpha]

In [ ]:
# Create a new variable denoting subjects who responded either "yes" or "unsure"
data["rand_mech_yesorunsure"] = data["rand_mech_response"] > 0

y = "ASR_Intn_T"

fig, ax = plt.subplots(figsize=(3,10))

#sns.set_palette(["tab:blue", "tab:red"])
ax = sns.boxplot(x="rand_mech_yesorunsure", y=y, data=data)
ax.set_xlabel("RANDMECH response")
ax.set_xticklabels(["\"Non-\nsocial\"", "\"Social\" or \n\"Unsure\""])

'''for patch in ax.artists:
 r, g, b, a = patch.get_facecolor()
 patch.set_facecolor((r, g, b, alpha))
 patch.set_edgecolor((r, g, b, 1))'''
 
for i,artist in enumerate(ax.artists):
    if i == 0:
        col = blue_rgb + [alpha]
    else:
        col = red_rgb + [alpha]
        # Set the linecolor on the artist to the facecolor, and set the facecolor to None
    #col = artist.get_facecolor()
    artist.set_edgecolor(col)
    #artist.set_edgecolor('None')
    artist.set_facecolor(list(col[:3])+[alpha])
    if np.round(col[0],2) ==  .75:
        print(i)
    # Each box has 6 associated Line2D objects (to make the whiskers, fliers, etc.)
    # Loop over them here, and use the same colour as above
    
    for j in range(i*6,i*6+6):
        line = ax.lines[j]
        if j in [4,10]:
            line.set_color('k')
            line.set_linestyle('--')
            line.set_linewidth(2)
        else:
            line.set_color(col)
        line.set_mfc(col)
        line.set_mec(col)
        if j in [5,11]:
            line.set_marker('o')
plt.xlabel('')
plt.title('RANDOM MECH', fontweight = 'bold')
t, p = stats.ttest_ind(data[data["rand_mech_yesorunsure"]==1][y], data[data["rand_mech_yesorunsure"]==0][y], nan_policy='omit')
ax.annotate(f't = {t:.2f},\n(p = {p:.2g})', xy=(0.05, 0.85), xycoords='axes fraction')

fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


In [ ]:
def box_plot(data, edge_color, fill_color,pos,v):
    # customize boxplots
    alpha= .2
    data = data[~np.isnan(data)]
    bp = ax.boxplot(data, positions = [pos], patch_artist=True,widths=.6,vert=v,flierprops = dict(markeredgecolor=edge_color,
    markerfacecolor=fill_color, alpha=alpha))
    
    for element in ['boxes', 'whiskers', 'fliers', 'means', 'caps']:
        plt.setp(bp[element], color=edge_color)
    for element in ['medians']:
        plt.setp(bp[element], color='k',linewidth=2,ls='dashed')

    for patch in bp['boxes']:
        patch.set(facecolor=fill_color,alpha=alpha)#'w'       
        
    return bp

In [ ]:
#data1 = data.loc[data['rand_mech_response']!=9,:]
#data1.shape
fig, ax = plt.subplots(figsize=(3,5))

y = "ASR_Intn_T"
#x = "rand_mech_response"
x = "rand_mech_yesorunsure"
rows = data[x] == 0
bp2 = box_plot(data.loc[rows,y], blue_rgb, blue_rgb,0,True)
rows = data[x] == 1
bp1 = box_plot(data.loc[rows,y], red_rgb, red_rgb,1,True)
plt.xticks(range(2),['"Non-social"','"Social"/\n "Unsure"'])
t, p = stats.ttest_ind(data[data[x]==1][y], data[data[x]==0][y], nan_policy='omit')
plt.ylabel('ASR_Intn_T')
ax.annotate(f't = {t:.2f}\n(p = {p:.2g})', xy=(0.05, 0.85), xycoords='axes fraction')
plt.title('RANDOM MECH',fontweight='bold')
fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


In [ ]:
# Create a new variable denoting subjects who responded either "yes" or "unsure"
data["rand_mech_yesorunsure"] = data["rand_mech_response"] > 0

y = "ASR_Intn_T"

fig, ax = plt.subplots(figsize=(5,10))

sns.set_palette(["tab:blue", "tab:red"])
ax = sns.boxplot(x="rand_mech_yesorunsure", y=y, data=data)
ax.set_xlabel("RANDMECH response")
ax.set_xticklabels(["\"Non-social\"", "\"Social\" or \n\"Unsure\""])

t, p = stats.ttest_ind(data[data["rand_mech_yesorunsure"]==1][y], data[data["rand_mech_yesorunsure"]==0][y], nan_policy='omit')
ax.annotate(f't = {t:.2f} (p = {p:.2g})', xy=(0.05, 0.9), xycoords='axes fraction')

#fig.savefig('../results/RANDMECHresp_vs_ASRIntnT.png')


Look at RT on RANDMECH:

In [ ]:
data = data.join(all_trial_data[all_trial_data["movie"]=="Random mechanical.AVI"]["rt"])
data = data.rename(columns={"rt": "rand_mech_rt"})
data.head()

In [ ]:
g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data)

In [ ]:
# Restrict correlation to only subjects that responded "yes" or "unsure"

g = explore_correlation(x="rand_mech_rt", y="ASR_Intn_T", data=data[data["rand_mech_yesorunsure"]==1])